## Processing data for chloropleth maps

Notebook to perform some simple data transformations and joins to produce data frames for electoral area and electoral ward property price chloropleth maps.

In [53]:
import pandas as pd
import geopandas as gpd

electoral_area_annual = pd.ExcelFile(
    "~/Downloads/District Electoral Area Annual Price Statistics Property Types_Frozen.xlsx"
)

ward_annual = pd.ExcelFile("~/Downloads/Electoral Ward Annual Price Statistics Property Types_Frozen.xlsx")


In [115]:
def get_all_districts_for_all_house_types(electoral_area_annual, area_type="district"):

    area_type_mapping = {
        "district": "District Electoral Area (2014)",
        "ward": "Electoral Ward (2014)"
    }

    data = (
        pd.concat(
            [
                get_all_districts_data(electoral_area_annual, i, key=area_type_mapping[area_type])
                for i in ["Total", "DET", "SDT", "TER", "Apt"]
            ],
            axis=1
        )
        .reset_index(drop=True)
    )
    return data.loc[:, ~data.columns.duplicated()].copy()


def get_all_districts_data(excel, house_type, key="District Electoral Area (2014)"):
    district_sheets = [i for i in excel.sheet_names if house_type in i]
    list_of_dfs = []
    for sheet in district_sheets:
        df = pd.read_excel(excel, sheet)
        df = df.drop([0, 1, 2]).reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop([0])
        list_of_dfs.append(get_data_for_one_district(df, key))

    all_data  = pd.concat(list_of_dfs).reset_index(drop=True)
    all_data.rename({"Median Sale Price": f"median_sale_price_{house_type}".lower()}, inplace=True, axis=1)
    return all_data


def get_data_for_one_district(df, key):
    grouped = df.groupby(key, as_index=False)["Median Sale Price"].last()
    grouped["Median Sale Price"] = grouped["Median Sale Price"].apply(lambda x: price_to_numeric(x))
    return grouped

def price_to_numeric(price):
    if price == ".":
        return float("nan")
    else:
        return int(price.replace("£", "").replace(",", ""))



In [116]:
int("£300,000".replace("£", ""))

ValueError: invalid literal for int() with base 10: '300,000'

In [117]:
price_data_districts = get_all_districts_for_all_house_types(electoral_area_annual)
price_data_wards = get_all_districts_for_all_house_types(ward_annual, area_type="ward")

In [118]:
price_data_districts

,District Electoral Area (2014),median_sale_price_total,median_sale_price_det,median_sale_price_sdt,median_sale_price_ter,median_sale_price_apt
0,Airport,185000,270000.0,187500.0,144000.0,120000.0
1,Antrim,141000,234311.0,162000.0,99500.0,101000.0
2,Ballyclare,171000,250000.0,174500.0,105000.0,NaN
3,Dunsilly,172500,226000.0,160000.0,120000.0,NaN
4,Glengormley Urban,160000,200000.0,160000.0,97000.0,NaN
...,...,...,...,...,...,...
75,Newry,150000,255000.0,175000.0,120000.0,NaN
76,Rowallane,170000,240000.0,165000.0,129000.0,NaN
77,Slieve Croob,182000,266000.0,172000.0,142500.0,NaN
78,Slieve Gullion,158000,237568.0,156000.0,119000.0,NaN


In [119]:
gdf_electoral_areas = gpd.read_file("../data/shapefiles/OSNI_Open_Data_-_Largescale_Boundaries_-_District_Electoral_Areas_(2012).geojson")

gdf_electoral_areas = gdf_electoral_areas.rename_geometry("WKT")
gdf_electoral_areas["FinalR_DEA"] = gdf_electoral_areas["FinalR_DEA"].apply(lambda x: " ".join([i.capitalize() for i in x.split(" ")]))

electoral_area_chloropleth_data = gdf_electoral_areas.merge(
    price_data_districts, left_on="FinalR_DEA", right_on="District Electoral Area (2014)", how="outer"
)

In [120]:
electoral_area_chloropleth_data.to_csv("../data/chloropleth_data/electoral_areas.csv", index=False)

In [121]:
gdf_wards = gpd.read_file("../data/shapefiles/OSNI_Open_Data_-_Largescale_Boundaries_-_Wards_(2012).geojson")

gdf_wards = gdf_wards.rename_geometry("WKT")
gdf_wards["WARDNAME"] = gdf_wards["WARDNAME"].apply(lambda x: " ".join([i.capitalize() for i in x.split(" ")]))

electoral_ward_chloropleth_data = gdf_wards.merge(
    price_data_wards, left_on="WARDNAME", right_on="Electoral Ward (2014)", how="outer"
)

In [122]:
electoral_ward_chloropleth_data.to_csv("../data/chloropleth_data/electoral_wards.csv", index=False)

In [123]:
gpd.read_file("../data/chloropleth_data/electoral_wards.csv")

,OBJECTID,WARDNAME,WardCode,SHAPE_Length,SHAPE_Area,WKT,Electoral Ward (2014),median_sale_price_total,median_sale_price_det,median_sale_price_sdt,median_sale_price_ter,median_sale_price_apt,geometry
0,1.0,Abbey,N08000101,5294.736243702003,910177.7709385309,POLYGON ((-5.904608242956746 54.67168352755852...,Abbey,125000.0,,,115000.0,,"POLYGON ((-5.90461 54.67168, -5.90458 54.67179..."
1,1.0,Abbey,N08000101,5294.736243702003,910177.7709385309,POLYGON ((-5.904608242956746 54.67168352755852...,Abbey,120000.0,,,,,"POLYGON ((-5.90461 54.67168, -5.90458 54.67179..."
2,329.0,Abbey,N08001001,8271.762558930823,3295932.6361528607,POLYGON ((-6.339341401587046 54.17657758586186...,Abbey,125000.0,,,115000.0,,"POLYGON ((-6.33934 54.17658, -6.33940 54.17651..."
3,329.0,Abbey,N08001001,8271.762558930823,3295932.6361528607,POLYGON ((-6.339341401587046 54.17657758586186...,Abbey,120000.0,,,,,"POLYGON ((-6.33934 54.17658, -6.33940 54.17651..."
4,251.0,Academy,N08000801,8046.440397365479,2609260.980243188,POLYGON ((-6.311956684255974 54.85847575432261...,Academy,187500.0,,,,,"POLYGON ((-6.31196 54.85848, -6.31038 54.85847..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,128.0,Windsor,N08000358,5698.221941366609,1381021.7091559132,POLYGON ((-5.936280216926629 54.58517356478286...,Windsor,165000.0,,,160000.0,148000.0,"POLYGON ((-5.93628 54.58517, -5.93616 54.58552..."
492,169.0,Windy Hall,N08000440,8606.892613604005,2914845.179747041,"POLYGON ((-6.635208431485175 55.1028144182293,...",Windy Hall,179950.0,235000.0,,,,"POLYGON ((-6.63521 55.10281, -6.63408 55.10319..."
493,289.0,Woodburn,N08000840,38678.807968719855,49065453.04116042,POLYGON ((-5.899706613345378 54.70851929294969...,Woodburn,174475.0,,,,,"POLYGON ((-5.89971 54.70852, -5.89958 54.70852..."
494,129.0,Woodstock,N08000359,4003.7772545756598,666442.4015081623,POLYGON ((-5.903001532623888 54.59112170900987...,Woodstock,124375.0,0.0,,122750.0,,"POLYGON ((-5.90300 54.59112, -5.90415 54.59235..."


In [72]:
electoral_ward_chloropleth_data.columns

Index(['OBJECTID', 'WARDNAME', 'WardCode', 'SHAPE_Length', 'SHAPE_Area',
       'geometry', 'Electoral Ward (2014)', 'median_sale_price_total',
       'median_sale_price_det', 'median_sale_price_sdt',
       'median_sale_price_ter', 'median_sale_price_apt'],
      dtype='object')